<a href="https://colab.research.google.com/github/Vasilis-Kyriakopoulos/Finetune-LLM-/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
import pandas as pd


ds = load_dataset("maxscha/commitbench")
main_path = "/content/drive/MyDrive/FinetuneLLM_Ergasia/"

cols = ["diff", "message"]


ds["train"].select_columns(cols) \
    .to_pandas() \
    .to_csv(main_path+"commitbench_train.csv", index=False)


ds["validation"].select_columns(cols) \
    .to_pandas() \
    .to_csv(main_path+"commitbench_validation.csv",index=False)


ds["test"].select_columns(cols) \
    .to_pandas() \
    .to_csv(main_path+"commitbench_test.csv", index=False)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/225M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/225M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
from google.colab import drive
import pandas as pd


drive.mount('/content/drive')
main_path = "/content/drive/MyDrive/FinetuneLLM_Ergasia/"
train_df = pd.read_csv(main_path + "commitbench_train_1pct.csv")
val_df = pd.read_csv(main_path + "commitbench_validation_1pct.csv")

Mounted at /content/drive


In [6]:
import pandas as pd

input_csv = main_path + "commitbench_validation.csv"
output_csv = main_path + "commitbench_validation_1pct.csv"

chunk_size = 100_000
keep_frac = 0.01
first = True

for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
    sampled = chunk.sample(frac=keep_frac, random_state=42)

    sampled.to_csv(
        output_csv,
        mode="a",
        index=False,
        header=first
    )
    first = False


In [4]:
train_df.head()

,diff,message
0,diff --git a/lib/is_translatable.rb b/lib/is_t...,Adding some failing specs to get the translati...
1,diff --git a/src/Event.php b/src/Event.php\nin...,Added getDescription function to Event
2,diff --git a/templates/module/modularity-mod-s...,Dont initialize slider if column count is same...
3,diff --git a/src/FieldHandlers/BaseFileFieldHa...,letting the user open a file in the new tab (t...
4,diff --git a/bids/layout/tests/test_layout.py ...,TEST: Test upward propagation of dynamic gette...


In [5]:
val_df.head()

,diff,message
0,diff --git a/lib/dicom/anonymizer.rb b/lib/dic...,Added Anonymizer#to_anonymizer
1,diff --git a/comments-bundle/contao/Comments.p...,[Comments] Do not allow insert tags in comment...
2,diff --git a/src/MetaBox.php b/src/MetaBox.php...,Use specific types annotations for arrays in d...
3,diff --git a/header.js b/header.js\nindex <HAS...,"Fix lint, rlp isn't used"
4,diff --git a/taskw/warrior.py b/taskw/warrior....,"Merge the ""waiting"" list back into the ""pendin..."


diff       diff --git a/lib/is_translatable.rb b/lib/is_t...
message    Adding some failing specs to get the translati...
Name: 0, dtype: object
diff --git a/lib/is_translatable.rb b/lib/is_translatable.rb
index <HASH>..<HASH> 100644
--- a/lib/is_translatable.rb
+++ b/lib/is_translatable.rb
@@ -13,6 +13,9 @@ module IsTranslatable
 
     def set_translation(kind, t, locale_override=nil)
     end
+
+	def get_translation(kind, locale_override=nil)
+	end
   end
 end
 
diff --git a/spec/lib/is_translatable_spec.rb b/spec/lib/is_translatable_spec.rb
index <HASH>..<HASH> 100644
--- a/spec/lib/is_translatable_spec.rb
+++ b/spec/lib/is_translatable_spec.rb
@@ -24,7 +24,20 @@ describe IsTranslatable do
       context 'with translated title' do
         before {@article.set_translation(:title, @titles[:es])}
         it {should be_valid}
-        it "should check that it's actually translated"
+
+        it {subject.get_translation(:title).should == @titles[:es]}
+
+		context 'loaded from db' do


In [ ]:
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


def token_len(t: str) -> int:
    return len(tokenizer.encode(t, add_special_tokens=True))


rng = np.random.default_rng(42)
texts = train_df.apply(format_input, axis=1).tolist()
sample_size = min(50_000, len(texts))
sample_texts = rng.choice(texts, size=sample_size, replace=False)

lengths = np.array([token_len(t) for t in sample_texts], dtype=np.int32)

p = [50, 75, 90, 95, 97, 98, 99, 99.5]
qs = np.percentile(lengths, p)

for perc, q in zip(p, qs):
    print(f"{perc:>5}%  -> {int(q)} tokens")

# Example choice:
max_len = int(np.percentile(lengths, 95))
print("Suggested max_len:", max_len)


Token indices sequence length is longer than the specified maximum sequence length for this model (1069 > 1024). Running this sequence through the model will result in indexing errors


   50%  -> 327 tokens
   75%  -> 427 tokens
   90%  -> 525 tokens
   95%  -> 587 tokens
   97%  -> 634 tokens
   98%  -> 668 tokens
   99%  -> 734 tokens
 99.5%  -> 789 tokens
Suggested max_len: 587


In [6]:
import torch
from torch.utils.data import Dataset
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))
def format_input(entry):

    input_text = f"{entry['diff']}\nCommit Message:\n{entry['message']}"

    return input_text
print(train_df.iloc[0])
print(format_input(train_df.iloc[0]))


class CodeDiffMessageDataset(Dataset):
    def __init__(self, data, tokenizer,max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = 1024


    def __getitem__(self, index):
        self.encoded_text = None
        entry = self.data.iloc[index]
        code_diff_plus_message = format_input(entry)
        self.encoded_text = tokenizer.encode(code_diff_plus_message,max_length=self.max_length)
        return self.encoded_text

    def __len__(self):
        return len(self.data)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)

def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index


        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    device = None
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Device:", device)

[50256]
diff       diff --git a/lib/is_translatable.rb b/lib/is_t...
message    Adding some failing specs to get the translati...
Name: 0, dtype: object
diff --git a/lib/is_translatable.rb b/lib/is_translatable.rb
index <HASH>..<HASH> 100644
--- a/lib/is_translatable.rb
+++ b/lib/is_translatable.rb
@@ -13,6 +13,9 @@ module IsTranslatable
 
     def set_translation(kind, t, locale_override=nil)
     end
+
+	def get_translation(kind, locale_override=nil)
+	end
   end
 end
 
diff --git a/spec/lib/is_translatable_spec.rb b/spec/lib/is_translatable_spec.rb
index <HASH>..<HASH> 100644
--- a/spec/lib/is_translatable_spec.rb
+++ b/spec/lib/is_translatable_spec.rb
@@ -24,7 +24,20 @@ describe IsTranslatable do
       context 'with translated title' do
         before {@article.set_translation(:title, @titles[:es])}
         it {should be_valid}
-        it "should check that it's actually translated"
+
+        it {subject.get_translation(:title).should == @titles[:es]}
+
+		context 'loaded from

In [7]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_fn(batch))

(tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]], device='cuda:0'), tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]], device='cuda:0'))


Device: cuda


In [8]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 4

torch.manual_seed(123)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

In [ ]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Train loader:
torch.Size([16, 853]) torch.Size([16, 853])
torch.Size([16, 587]) torch.Size([16, 587])
torch.Size([16, 584]) torch.Size([16, 584])
torch.Size([16, 526]) torch.Size([16, 526])
torch.Size([16, 534]) torch.Size([16, 534])
torch.Size([16, 587]) torch.Size([16, 587])
torch.Size([16, 729]) torch.Size([16, 729])
torch.Size([16, 594]) torch.Size([16, 594])
torch.Size([16, 866]) torch.Size([16, 866])
torch.Size([16, 449]) torch.Size([16, 449])
torch.Size([16, 596]) torch.Size([16, 596])
torch.Size([16, 665]) torch.Size([16, 665])
torch.Size([16, 725]) torch.Size([16, 725])
torch.Size([16, 594]) torch.Size([16, 594])
torch.Size([16, 710]) torch.Size([16, 710])
torch.Size([16, 521]) torch.Size([16, 521])
torch.Size([16, 532]) torch.Size([16, 532])
torch.Size([16, 601]) torch.Size([16, 601])
torch.Size([16, 683]) torch.Size([16, 683])
torch.Size([16, 727]) torch.Size([16, 727])
torch.Size([16, 590]) torch.Size([16, 590])
torch.Size([16, 698]) torch.Size([16, 698])
torch.Size([16, 61

Token indices sequence length is longer than the specified maximum sequence length for this model (1262 > 1024). Running this sequence through the model will result in indexing errors


torch.Size([16, 713]) torch.Size([16, 713])
torch.Size([16, 686]) torch.Size([16, 686])
torch.Size([16, 511]) torch.Size([16, 511])
torch.Size([16, 768]) torch.Size([16, 768])
torch.Size([16, 638]) torch.Size([16, 638])
torch.Size([16, 556]) torch.Size([16, 556])
torch.Size([16, 646]) torch.Size([16, 646])
torch.Size([16, 1262]) torch.Size([16, 1262])
torch.Size([16, 508]) torch.Size([16, 508])
torch.Size([16, 558]) torch.Size([16, 558])
torch.Size([16, 698]) torch.Size([16, 698])
torch.Size([16, 683]) torch.Size([16, 683])
torch.Size([16, 651]) torch.Size([16, 651])
torch.Size([16, 480]) torch.Size([16, 480])
torch.Size([16, 536]) torch.Size([16, 536])
torch.Size([16, 579]) torch.Size([16, 579])
torch.Size([16, 693]) torch.Size([16, 693])
torch.Size([16, 672]) torch.Size([16, 672])
torch.Size([16, 565]) torch.Size([16, 565])
torch.Size([16, 585]) torch.Size([16, 585])
torch.Size([16, 589]) torch.Size([16, 589])
torch.Size([16, 607]) torch.Size([16, 607])
torch.Size([16, 559]) torch.Si

In [ ]:

print(inputs[5])

tensor([26069,  1377, 18300,   257,    14, 41989,    14,  7282,   437,    14,
        17227,  7841,    16,    14,    33,  1347,  1039, 14402,    13, 10121,
          275,    14, 41989,    14,  7282,   437,    14, 17227,  7841,    16,
           14,    33,  1347,  1039, 14402,    13, 10121,   198,  9630,  1279,
           39, 11211,    29,   492,    27,    39, 11211,    29,  1802, 29173,
          198,  6329,   257,    14, 41989,    14,  7282,   437,    14, 17227,
         7841,    16,    14,    33,  1347,  1039, 14402,    13, 10121,   198,
        45340,   275,    14, 41989,    14,  7282,   437,    14, 17227,  7841,
           16,    14,    33,  1347,  1039, 14402,    13, 10121,   198, 12404,
          532,  1899,    11,    21,  1343,  1899,    11,    23, 25248,  1398,
        13452,  1039, 14402, 14582, 20514,    32, 14415, 14402, 20448,   198,
          220,   198,   220,   220,   220,   220,   220,   220,   220,   220,
          720,  5661,  3784, 30493, 23588,   874, 16763, 13812, 

In [ ]:

print(targets[5])

tensor([ 1377, 18300,   257,    14, 41989,    14,  7282,   437,    14, 17227,
         7841,    16,    14,    33,  1347,  1039, 14402,    13, 10121,   275,
           14, 41989,    14,  7282,   437,    14, 17227,  7841,    16,    14,
           33,  1347,  1039, 14402,    13, 10121,   198,  9630,  1279,    39,
        11211,    29,   492,    27,    39, 11211,    29,  1802, 29173,   198,
         6329,   257,    14, 41989,    14,  7282,   437,    14, 17227,  7841,
           16,    14,    33,  1347,  1039, 14402,    13, 10121,   198, 45340,
          275,    14, 41989,    14,  7282,   437,    14, 17227,  7841,    16,
           14,    33,  1347,  1039, 14402,    13, 10121,   198, 12404,   532,
         1899,    11,    21,  1343,  1899,    11,    23, 25248,  1398, 13452,
         1039, 14402, 14582, 20514,    32, 14415, 14402, 20448,   198,   220,
          198,   220,   220,   220,   220,   220,   220,   220,   220,   720,
         5661,  3784, 30493, 23588,   874, 16763, 13812,  5376, 

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = False
model.config.pad_token_id = model.config.eos_token_id
print(tokenizer.vocab_size)
model.config.pad_token_id





/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

50257


50256

In [ ]:
print(model.config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.3",
  "use_cache": false,
  "vocab_size": 50257
}



In [10]:
torch.cuda.empty_cache()
import gc
gc.collect()


215

In [ ]:
import torch
import torch.nn as nn
import time
from transformers import pipeline
from tqdm.auto import tqdm


def generate():
  # --- Inside your training loop ---
# 1. Use a real 'diff' from your data if possible, or a formatted string
  example_diff = "--- a/file.py\n+++ b/file.py\n- print('hi')\n+ print('hello')"
  prompt = f"Diff:\n{example_diff}\n\nCommit Message:\n"

# 2. Encode with attention mask
  inputs = tokenizer(
    prompt,
    return_tensors="pt",
    max_length=512,
    padding=True,
    truncation=True
  ).to(device)

# 3. Generate with explicit config
  with torch.no_grad():
      outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"], # Crucial for reliability
        max_new_tokens=50,                       # Generates 50 new tokens BEYOND the prompt
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# 4. Decode (Skip the prompt to see only the message)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(f"Generated Result:\n{generated_text}")

def train_one_epoch(model, dataloader, optimizer, device,freq_print):
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")

    for i, (x, y) in progress_bar:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        torch.cuda.empty_cache()

        logits = model(x).logits                    # (B, T, vocab)
        loss = nn.CrossEntropyLoss()(
            logits.view(-1, logits.size(-1)),
            y.view(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})
        if i % 20 ==0:
            print(f"steps:{i+1}/{len(dataloader)}")
            generate()
        torch.cuda.empty_cache()

    return total_loss / len(dataloader)


def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x).logits
            loss = nn.CrossEntropyLoss()(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_loss += loss.item()

    return total_loss / len(dataloader)


def train(model, train_loader, optimizer, device):

    # -----------------------------
    # Training Loop
    # -----------------------------
    best_val_loss = float("inf")
    patience = 2            # stop after 2 bad epochs
    bad_epochs = 0
    epochs = 2
    train_losses, val_losses = [],[]

    for epoch in range(epochs):

        train_loss = train_one_epoch(model, train_loader, optimizer, device,freq_print=1)
        #val_loss = evaluate(model, val_loader, device)
        train_losses.append(train_loss)
        #val_losses.append(val_loss)
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss:   {val_loss:.4f}")
        torch.save(model.state_dict(), f"models/gpt2_epoch{epoch+1}.pt")
        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            bad_epochs = 0
            torch.save(model.state_dict(), main_path+"models/best_model.pt")
            print("New best model saved!")
        else:
            bad_epochs += 1
            print(f"No improvement (bad epochs: {bad_epochs})")

        if bad_epochs >= patience:
            print("EARLY STOPPING TRIGGERED.")
            break

    epochs_tensor = torch.linspace(0, epochs, len(train_losses))
    for i,(val_loss,train_loss)  in enumerate(zip(val_losses,train_losses)):
        print(f"Epoch {i+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
train(model, train_loader, optimizer, device)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


steps:1/3163


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Generated Result:
Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:



Hi, I'm sorry for the inconvenience, but I don't know what to do with this. I've been working on this for a long time, so I thought I'd give it a try. It's been a while since I
steps:21/3163
Generated Result:
Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fixes a bug that could cause a crash if a file was created with a wrong name.
steps:41/3163
Generated Result:
Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
This is a new version of the module.

steps:61/3163
Generated Result:
Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fixes #<I>
steps:81/3163
Generated Result:
Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fix a bug in the file.
steps:101/3163
Generated Result:
Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message: